In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# Any results you write to the current directory are saved as output.
# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
train = pd.read_csv('data/train.csv.zip', low_memory=False)
test = pd.read_csv('data/test.csv.zip', low_memory=False)
submission = pd.read_csv('data/sample_submission.csv.zip', low_memory=False)

In [6]:
print(train.info())
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 857895 entries, 0 to 857894
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   overall         857895 non-null  float64
 1   verified        857895 non-null  bool   
 2   reviewTime      857895 non-null  object 
 3   asin            857895 non-null  object 
 4   reviewerName    857719 non-null  object 
 5   reviewText      857616 non-null  object 
 6   summary         857725 non-null  object 
 7   unixReviewTime  857895 non-null  int64  
 8   vote            118740 non-null  object 
 9   style           444406 non-null  object 
 10  image           7142 non-null    object 
 11  userid          857895 non-null  int64  
 12  itemid          857895 non-null  int64  
 13  rating          857895 non-null  float64
dtypes: bool(1), float64(2), int64(3), object(8)
memory usage: 85.9+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285965 entries, 0 to 28596

In [7]:
def describe(df, field_name):
    print(f"Колонка \"{field_name}\":")
    print("------")
    print("na:", df[field_name].isna().sum())
    print("уникальных значений:", len(df[field_name].unique()))
    print("------")
    print(df[field_name].value_counts())  

In [8]:
for field_name in train.columns.tolist():
    describe(train, field_name)

Колонка "overall":
------
na: 0
уникальных значений: 5
------
5.0    615196
4.0    113094
3.0     60440
1.0     37498
2.0     31667
Name: overall, dtype: int64
Колонка "verified":
------
na: 0
уникальных значений: 2
------
True     745243
False    112652
Name: verified, dtype: int64
Колонка "reviewTime":
------
na: 0
уникальных значений: 4580
------
03 1, 2016     942
07 14, 2016    885
06 3, 2015     799
01 20, 2016    795
02 20, 2016    784
              ... 
12 14, 2003      1
02 25, 2005      1
07 13, 2005      1
09 16, 2005      1
03 25, 2006      1
Name: reviewTime, Length: 4580, dtype: int64
Колонка "asin":
------
na: 0
уникальных значений: 41302
------
B00BUKL666    5491
B00D3M2QP4    4733
B008QMX2SG    4654
B00R7PWK7W    2547
B000X3TPHS    2300
              ... 
B005CJUXOM       1
B005TY2FQY       1
B003VTN9VE       1
B00HFVVEPW       1
B00NFOGLQA       1
Name: asin, Length: 41302, dtype: int64
Колонка "reviewerName":
------
na: 176
уникальных значений: 101208
------
Amazon C

In [9]:
import scipy.sparse as sparse

from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k
import sklearn
from sklearn.model_selection import train_test_split

import scipy.sparse as sparse

In [10]:
train_data, test_data = train_test_split(train,random_state=32, shuffle=True)


In [11]:
describe(train_data, 'rating')

Колонка "rating":
------
na: 0
уникальных значений: 2
------
1.0    546442
0.0     96979
Name: rating, dtype: int64


In [12]:
ratings_coo = sparse.coo_matrix((train_data['rating'].astype(int),
                                 (train_data['userid'],
                                  train_data['itemid'])))

In [13]:
NUM_THREADS = 4 #число потоков
NUM_COMPONENTS = 30 #число параметров вектора 
NUM_EPOCHS = 20 #число эпох обучения

model = LightFM(learning_rate=0.1, loss='logistic',
                no_components=NUM_COMPONENTS)
model = model.fit(ratings_coo, epochs=NUM_EPOCHS, 
                  num_threads=NUM_THREADS)

In [14]:
preds = model.predict(test_data.userid.values,
                      test_data.itemid.values)


In [15]:
sklearn.metrics.roc_auc_score(test_data.rating,preds)


0.7453574642585319

In [16]:
preds = model.predict(test.userid.values,
                      test.itemid.values)


In [17]:
preds.min(), preds.max()

(-22.84771728515625, 25.60747718811035)

In [18]:
normalized_preds = (preds - preds.min())/(preds - preds.min()).max()


In [19]:
normalized_preds.min(), normalized_preds.max()


(0.0, 1.0)

In [20]:
submission['rating']= normalized_preds

In [21]:
submission.to_csv('submission_log.csv', index=False)

In [22]:
submission

,Id,rating
0,0,0.516811
1,1,0.507286
2,2,0.483964
3,3,0.473054
4,4,0.514962
...,...,...
285960,285960,0.512952
285961,285961,0.378034
285962,285962,0.449119
285963,285963,0.513838
